# Fetch SPX Options Data from OpenBB

This notebook fetches SPX options data from OpenBB Platform and transforms it to WRDS-compatible format.

## 1. Setup and Imports

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime

# Import OpenBB
from openbb import obb

# Set output type to dataframe to avoid stream decoding errors
obb.user.preferences.output_type = "dataframe"
print("OpenBB output type set to: dataframe")
print("OpenBB initialized successfully!")

OpenBB output type set to: dataframe
OpenBB initialized successfully!


## 2. Fetch SPX Options Data from OpenBB (CBOE Provider)

In [2]:
# Fetch SPX options chain
print("Fetching SPX options data from OpenBB...")
print("Provider: CBOE (free tier)")

# Fetch data - will return dataframe directly
df_openbb = obb.derivatives.options.chains(symbol="SPX", provider="cboe")

print(f"\nFetched {len(df_openbb)} option quotes")
print(f"\nOpenBB DataFrame columns: {list(df_openbb.columns)}")
print(f"\nFirst few rows:")
df_openbb.head()

Fetching SPX options data from OpenBB...
Provider: CBOE (free tier)

Fetched 26212 option quotes

OpenBB DataFrame columns: ['underlying_symbol', 'underlying_price', 'contract_symbol', 'expiration', 'dte', 'strike', 'option_type', 'open_interest', 'volume', 'theoretical_price', 'last_trade_price', 'last_trade_time', 'tick', 'bid', 'bid_size', 'ask', 'ask_size', 'open', 'high', 'low', 'prev_close', 'change', 'change_percent', 'implied_volatility', 'delta', 'gamma', 'theta', 'vega', 'rho']

First few rows:


,underlying_symbol,underlying_price,contract_symbol,expiration,dte,strike,option_type,open_interest,volume,theoretical_price,...,low,prev_close,change,change_percent,implied_volatility,delta,gamma,theta,vega,rho
0,SPXW,6901.0,SPXW251211C02800000,2025-12-11,0,2800.0,call,0,4,4100.6745,...,4040.2,4095.349976,-41.35,-0.010097,0.0,1.0,0.0,0.0,0.0,0.0
1,SPXW,6901.0,SPXW251211P02800000,2025-12-11,0,2800.0,put,62,0,0.0000,...,0.0,0.025000,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,SPXW,6901.0,SPXW251211C03000000,2025-12-11,0,3000.0,call,0,0,3900.6745,...,0.0,3895.349976,0.00,0.000000,0.0,1.0,0.0,0.0,0.0,0.0
3,SPXW,6901.0,SPXW251211P03000000,2025-12-11,0,3000.0,put,0,0,0.0000,...,0.0,0.025000,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,SPXW,6901.0,SPXW251211C03200000,2025-12-11,0,3200.0,call,0,0,3700.6745,...,0.0,3695.550049,0.00,0.000000,0.0,1.0,0.0,0.0,0.0,0.0


## 3. Inspect OpenBB Data Structure

In [3]:
# Display data info
print("OpenBB Data Info:")
print(df_openbb.info())
print("\n" + "="*80)
print("Sample data:")
print(df_openbb.head(10))

OpenBB Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26212 entries, 0 to 26211
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   underlying_symbol   26212 non-null  object 
 1   underlying_price    26212 non-null  float64
 2   contract_symbol     26212 non-null  object 
 3   expiration          26212 non-null  object 
 4   dte                 26212 non-null  int64  
 5   strike              26212 non-null  float64
 6   option_type         26212 non-null  object 
 7   open_interest       26212 non-null  int64  
 8   volume              26212 non-null  int64  
 9   theoretical_price   26212 non-null  float64
 10  last_trade_price    26212 non-null  float64
 11  last_trade_time     20331 non-null  object 
 12  tick                26212 non-null  object 
 13  bid                 26212 non-null  float64
 14  bid_size            26212 non-null  int64  
 15  ask                 26212 non-null 

## 4. Transform to WRDS Format

### Required WRDS Columns:
- `date` - Quote date
- `exdate` - Expiration date
- `strike_price` - Strike × 1000 (CRITICAL!)
- `cp_flag` - 'C' or 'P'
- `best_bid` - Bid price
- `best_offer` - Ask price
- `am_settlement` - Always 1

In [4]:
print("Transforming to WRDS format...")

# Create WRDS-compatible DataFrame
df_wrds = pd.DataFrame()

# Map quote date - use today's date
quote_date = datetime.now().strftime('%Y-%m-%d')
df_wrds['date'] = [quote_date] * len(df_openbb)

# Map expiration date
df_wrds['exdate'] = pd.to_datetime(df_openbb['expiration'])

# Map strike price - CRITICAL: Multiply by 1000!
df_wrds['strike_price'] = df_openbb['strike'] * 1000

# Map option type to 'C' or 'P'
df_wrds['cp_flag'] = df_openbb['option_type'].str.upper().str[0]  # 'call' -> 'C', 'put' -> 'P'

# Map bid and ask prices
df_wrds['best_bid'] = df_openbb['bid']
df_wrds['best_offer'] = df_openbb['ask']

# Set am_settlement to 1 (required by WRDS format)
df_wrds['am_settlement'] = 1

print(f"\nTransformed {len(df_wrds)} rows to WRDS format")
print(f"\nWRDS DataFrame columns: {list(df_wrds.columns)}")

Transforming to WRDS format...

Transformed 26212 rows to WRDS format

WRDS DataFrame columns: ['date', 'exdate', 'strike_price', 'cp_flag', 'best_bid', 'best_offer', 'am_settlement']


## 5. Data Quality Checks

In [5]:
print("Data Quality Checks:")
print("=" * 80)

# Check for missing values
print("\n1. Missing values:")
print(df_wrds.isnull().sum())

# Check option types
print("\n2. Option types:")
print(df_wrds['cp_flag'].value_counts())

# Check for zero or negative prices
print("\n3. Zero or negative bid prices:")
print(f"   Count: {(df_wrds['best_bid'] <= 0).sum()}")

print("\n4. Zero or negative ask prices:")
print(f"   Count: {(df_wrds['best_offer'] <= 0).sum()}")

# Check strike price range (after multiplication by 1000)
print("\n5. Strike price range:")
print(f"   Min: {df_wrds['strike_price'].min():.0f} (actual strike: {df_wrds['strike_price'].min()/1000:.2f})")
print(f"   Max: {df_wrds['strike_price'].max():.0f} (actual strike: {df_wrds['strike_price'].max()/1000:.2f})")

# Check expiration dates
print("\n6. Expiration dates:")
print(f"   Unique expiries: {df_wrds['exdate'].nunique()}")
print(f"   Date range: {df_wrds['exdate'].min()} to {df_wrds['exdate'].max()}")

# Count options per expiry (for put-call parity regression check)
print("\n7. Options per expiry:")
options_per_expiry = df_wrds.groupby('exdate').size()
print(f"   Min: {options_per_expiry.min()}")
print(f"   Max: {options_per_expiry.max()}")
print(f"   Mean: {options_per_expiry.mean():.1f}")

# Check for ATM options (approximate)
print("\n8. Strike distribution (approximate ATM check):")
print(df_wrds.groupby('exdate')['strike_price'].describe()[['count', 'min', '50%', 'max']])

print("\n" + "="*80)
print("WRDS-formatted data preview:")
print(df_wrds.head(10))

Data Quality Checks:

1. Missing values:
date             0
exdate           0
strike_price     0
cp_flag          0
best_bid         0
best_offer       0
am_settlement    0
dtype: int64

2. Option types:
cp_flag
C    13106
P    13106
Name: count, dtype: int64

3. Zero or negative bid prices:
   Count: 1797

4. Zero or negative ask prices:
   Count: 324

5. Strike price range:
   Min: 200000 (actual strike: 200.00)
   Max: 12000000 (actual strike: 12000.00)

6. Expiration dates:
   Unique expiries: 54
   Date range: 2025-12-11 00:00:00 to 2030-12-20 00:00:00

7. Options per expiry:
   Min: 132
   Max: 1908
   Mean: 485.4

8. Strike distribution (approximate ATM check):
             count        min        50%         max
exdate                                              
2025-12-11   444.0  2800000.0  6642500.0   8800000.0
2025-12-12   592.0  2200000.0  6577500.0   8800000.0
2025-12-15   442.0  2800000.0  6670000.0   8800000.0
2025-12-16   442.0  2800000.0  6670000.0   8800000.0
2025

## 6. Clean Data (Remove Invalid Entries)

In [6]:
print("Cleaning data...")
print(f"Initial rows: {len(df_wrds)}")

# Remove rows with missing values
df_wrds_clean = df_wrds.dropna()
print(f"After removing NaN: {len(df_wrds_clean)}")

# Remove expired options (expiry date in the past)
df_wrds_clean['exdate'] = pd.to_datetime(df_wrds_clean['exdate'])
today = pd.Timestamp.now().normalize()
df_wrds_clean = df_wrds_clean[df_wrds_clean['exdate'] > today]
print(f"After removing expired options: {len(df_wrds_clean)}")

# Remove zero or negative prices
df_wrds_clean = df_wrds_clean[
    (df_wrds_clean['best_bid'] > 0) & 
    (df_wrds_clean['best_offer'] > 0)
]
print(f"After removing zero/negative prices: {len(df_wrds_clean)}")

# Ensure valid option types
df_wrds_clean = df_wrds_clean[df_wrds_clean['cp_flag'].isin(['C', 'P'])]
print(f"After validating option types: {len(df_wrds_clean)}")

# Sort by date, expiry, strike
df_wrds_clean = df_wrds_clean.sort_values(['date', 'exdate', 'strike_price'])

print(f"\nFinal clean data: {len(df_wrds_clean)} rows")
print(f"Unique expiries: {df_wrds_clean['exdate'].nunique()}")

Cleaning data...
Initial rows: 26212
After removing NaN: 26212
After removing expired options: 25768
After removing zero/negative prices: 24175
After validating option types: 24175

Final clean data: 24175 rows
Unique expiries: 53


## 7. Save to CSV

In [7]:
# Create output directory
output_dir = Path('../data/openbb/spx')
output_dir.mkdir(parents=True, exist_ok=True)

# Create filename with date
quote_date = df_wrds_clean['date'].iloc[0]
if isinstance(quote_date, str):
    date_str = quote_date
else:
    date_str = quote_date.strftime('%Y-%m-%d')
    
output_file = output_dir / f'spx_options_{date_str}.csv'

# Save to CSV
df_wrds_clean.to_csv(output_file, index=False)

print(f"\nSaved WRDS-compatible CSV to: {output_file}")
print(f"File size: {output_file.stat().st_size / 1024:.1f} KB")
print(f"\nTo use with existing code, set:")
print(f"  os.environ['OPDS_WRDS_DATA_DIR'] = '{output_dir.absolute()}'")


Saved WRDS-compatible CSV to: ../data/openbb/spx/spx_options_2025-12-11.csv
File size: 1118.5 KB

To use with existing code, set:
  os.environ['OPDS_WRDS_DATA_DIR'] = '/Users/anighot/Documents/Thesis_New/operator-deep-smoothing-for-implied-volatility-main/volatility_smoothing/data/../data/openbb/spx'


## 8. Verify CSV Format

In [8]:
# Read back the CSV to verify format
df_verify = pd.read_csv(output_file)

print("Verification - Reading CSV back:")
print("=" * 80)
print(f"\nColumns: {list(df_verify.columns)}")
print(f"\nExpected columns: ['date', 'exdate', 'strike_price', 'cp_flag', 'best_bid', 'best_offer', 'am_settlement']")
print(f"\nRows: {len(df_verify)}")
print(f"\nData types:")
print(df_verify.dtypes)
print(f"\nSample data:")
print(df_verify.head(10))


Verification - Reading CSV back:

Columns: ['date', 'exdate', 'strike_price', 'cp_flag', 'best_bid', 'best_offer', 'am_settlement']

Expected columns: ['date', 'exdate', 'strike_price', 'cp_flag', 'best_bid', 'best_offer', 'am_settlement']

Rows: 24175

Data types:
date              object
exdate            object
strike_price     float64
cp_flag           object
best_bid         float64
best_offer       float64
am_settlement      int64
dtype: object

Sample data:
         date      exdate  strike_price cp_flag  best_bid  best_offer  \
0  2025-12-11  2025-12-12     2200000.0       C    4699.5      4715.9   
1  2025-12-11  2025-12-12     2400000.0       C    4499.3      4517.8   
2  2025-12-11  2025-12-12     2600000.0       C    4299.4      4317.8   
3  2025-12-11  2025-12-12     2800000.0       C    4099.5      4117.8   
4  2025-12-11  2025-12-12     3000000.0       C    3899.5      3918.0   
5  2025-12-11  2025-12-12     3200000.0       C    3699.6      3718.0   
6  2025-12-11  2025-

## Summary

1. Fetched SPX options data from OpenBB
2. Transformed to exact WRDS format (including strike × 1000)
3. Cleaned and validated data
4. Saved to CSV in `volatility_smoothing/data/openbb/spx/`